# ANN implementation 

In [211]:
train_location = 'C:/Users/DELL PC/Desktop/Kaggle/Titanic/train.csv'
test_location = 'C:/Users/DELL PC/Desktop/Kaggle/Titanic/test.csv'
Original_test_location = 'C:/Users/DELL PC/Desktop/Kaggle/Titanic/testOriginal.csv'

In [212]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
import matplotlib as plt

In [213]:
trainData = pd.read_csv(train_location)
# dropping any row where age is nan because it is a huge factor
trainData = trainData.dropna(how='any',axis=0) 
X_train = trainData.iloc[:,:7].values
Y_train = trainData.iloc[:,7].values

In [ ]:
# You are passing a target array of shape (712, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` 
# expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you 
# can convert them to the expected format via: ie convert 1 and 0 to [1, 0] and [0, 1] respectively.

from keras.utils import to_categorical
Y_train = to_categorical(Y_train)

In [214]:
testData = pd.read_csv(test_location)
X_test = testData.iloc[:,:7].values

In [215]:
trainData.shape

(712, 8)

In [216]:
testData.shape

(418, 7)

In [217]:
trainData;

In [218]:
# dont drop test values as it is required to submit 418 rows of predictions
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
testImputer = imputer.fit(X_test[:,:])
X_test[:,:] = testImputer.transform(X_test[:,:])

In [219]:
X_train

array([[ 3.    ,  0.    , 22.    , ...,  0.    ,  7.25  ,  1.    ],
       [ 1.    ,  1.    , 38.    , ...,  0.    , 71.2833,  0.    ],
       [ 3.    ,  1.    , 26.    , ...,  0.    ,  7.925 ,  1.    ],
       ...,
       [ 1.    ,  1.    , 19.    , ...,  0.    , 30.    ,  1.    ],
       [ 1.    ,  0.    , 26.    , ...,  0.    , 30.    ,  0.    ],
       [ 3.    ,  0.    , 32.    , ...,  0.    ,  7.75  ,  2.    ]])

In [220]:
# scaling 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [221]:
X_train.shape

(712, 7)

In [222]:
X_train.dtype

dtype('float64')

# Building ANN

In [366]:
import keras
from keras import layers
from keras import models
from keras import optimizers

In [367]:
model = models.Sequential()

In [368]:
model.add(layers.Dense(4, input_shape=(7,), activation='relu'))
# the model now takes input of dimensions (*,7)
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [369]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_106 (Dense)            (None, 4)                 32        
_________________________________________________________________
dense_107 (Dense)            (None, 8)                 40        
_________________________________________________________________
dense_108 (Dense)            (None, 16)                144       
_________________________________________________________________
dense_109 (Dense)            (None, 2)                 34        
Total params: 250
Trainable params: 250
Non-trainable params: 0
_________________________________________________________________


In [378]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [379]:
Y_train;

In [380]:
model.fit(x=X_train, y=Y_train, epochs=75, batch_size=None, verbose=2, validation_split=0.1, shuffle=True)

Train on 640 samples, validate on 72 samples
Epoch 1/75
 - 1s - loss: 0.3923 - acc: 0.8391 - val_loss: 0.3737 - val_acc: 0.8194
Epoch 2/75
 - 0s - loss: 0.3886 - acc: 0.8391 - val_loss: 0.3796 - val_acc: 0.7917
Epoch 3/75
 - 0s - loss: 0.3885 - acc: 0.8391 - val_loss: 0.3772 - val_acc: 0.8056
Epoch 4/75
 - 0s - loss: 0.3879 - acc: 0.8375 - val_loss: 0.3764 - val_acc: 0.7917
Epoch 5/75
 - 0s - loss: 0.3875 - acc: 0.8422 - val_loss: 0.3756 - val_acc: 0.8056
Epoch 6/75
 - 0s - loss: 0.3881 - acc: 0.8438 - val_loss: 0.3752 - val_acc: 0.7778
Epoch 7/75
 - 0s - loss: 0.3879 - acc: 0.8391 - val_loss: 0.3765 - val_acc: 0.7917
Epoch 8/75
 - 0s - loss: 0.3869 - acc: 0.8406 - val_loss: 0.3765 - val_acc: 0.7917
Epoch 9/75
 - 0s - loss: 0.3867 - acc: 0.8391 - val_loss: 0.3745 - val_acc: 0.7917
Epoch 10/75
 - 0s - loss: 0.3881 - acc: 0.8391 - val_loss: 0.3776 - val_acc: 0.7917
Epoch 11/75
 - 0s - loss: 0.3861 - acc: 0.8406 - val_loss: 0.3759 - val_acc: 0.7917
Epoch 12/75
 - 0s - loss: 0.3867 - acc: 

KeyboardInterrupt: 

In [373]:
predictions = model.predict_classes(X_test)
predictions.shape

(418,)

In [374]:
testDataOriginal = pd.read_csv(Original_test_location)
testDataOriginal["PassengerId"];

In [375]:
results = pd.DataFrame({
    "PassengerId": testDataOriginal["PassengerId"],
    "Survived": predictions})

In [376]:
results.to_csv('titanic_ANN4.csv', index=False)

In [381]:
model.save('Model.h5',overwrite=True)

do a train test split and then test on validation set